<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/SEIRS_sazonal_comparacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.integrate import solve_ivp

# Seasonal beta function: higher in winter (southern hemisphere)
def beta_seasonal(t, beta_base, beta_variation):
    return beta_base * (1 + beta_variation * np.cos(2 * np.pi * (t / 365)))

# SEIRS Model with Frequency-Dependent Transmission (Equations of Differences)
def seirs_discrete(S0, E0, I0, R0, beta_base, beta_variation, sigma, gamma, omega, N, time_step, sim_time):
    steps = int(sim_time / time_step)
    S, E, I, R = [S0], [E0], [I0], [R0]
    time_points = np.linspace(0, sim_time, steps + 1)

    for i in range(steps):
        t = time_points[i]
        beta_t = beta_seasonal(t, beta_base, beta_variation)
        new_E = beta_t * (S[-1] * I[-1] / N) * time_step
        new_I = sigma * E[-1] * time_step
        new_R = gamma * I[-1] * time_step
        new_S = omega * R[-1] * time_step

        S.append(S[-1] - new_E + new_S)
        E.append(E[-1] + new_E - new_I)
        I.append(I[-1] + new_I - new_R)
        R.append(R[-1] + new_R - new_S)

    return np.array(I), time_points

# SEIRS Model with Frequency-Dependent Transmission (Equations of Differential)
def seirs_model(t, y, beta_base, beta_variation, sigma, gamma, omega, N):
    S, E, I, R = y
    beta_t = beta_seasonal(t, beta_base, beta_variation)
    dSdt = -beta_t * (S * I / N) + omega * R
    dEdt = beta_t * (S * I / N) - sigma * E
    dIdt = sigma * E - gamma * I
    dRdt = gamma * I - omega * R
    return [dSdt, dEdt, dIdt, dRdt]

def solve_seirs(beta_base, beta_variation, sigma, gamma, omega, N, S0, E0, I0, R0, time_step, sim_time):
    y0 = [S0, E0, I0, R0]
    t_span = [0, sim_time]
    t_eval = np.arange(0, sim_time + time_step, time_step)  # Avoid exceeding sim_time
    t_eval = t_eval[t_eval <= sim_time]  # Ensure t_eval is within bounds

    sol = solve_ivp(seirs_model, t_span, y0, args=(beta_base, beta_variation, sigma, gamma, omega, N), t_eval=t_eval)
    return sol.t, sol.y[2]

# Interactive Widgets
beta_base_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.3, description='Beta Base')
beta_variation_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.01, value=0.2, description='Beta Variation')
sigma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Sigma')
gamma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Gamma')
omega_slider = widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='Omega')
N_slider = widgets.IntSlider(min=100, max=10000, step=100, value=1000, description='Population')
E0_slider = widgets.IntSlider(min=1, max=500, step=1, value=10, description='Initial E')
I0_slider = widgets.IntSlider(min=0, max=500, step=1, value=10, description='Initial I')
R0_slider = widgets.IntSlider(min=0, max=500, step=1, value=0, description='Initial R')
sim_time_slider = widgets.IntSlider(min=50, max=7*365, step=50, value=365, description='Simulation Time')
discrete_step_slider = widgets.FloatSlider(min=0.5, max=1.5, step=0.1, value=1, description='Discrete Time Step')
continuous_step_slider = widgets.FloatSlider(min=0.02, max=1.0, step=0.02, value=0.1, description='Continuous Time Step')

def plot_seirs(beta_base, beta_variation, sigma, gamma, omega, N, E0, I0, R0, discrete_time_step, continuous_time_step, sim_time):
    S0 = N - E0 - I0 - R0

    # Solve the models
    I_discrete, t_discrete = seirs_discrete(S0, E0, I0, R0, beta_base, beta_variation, sigma, gamma, omega, N, discrete_time_step, sim_time)
    t_diff, I_diff = solve_seirs(beta_base, beta_variation, sigma, gamma, omega, N, S0, E0, I0, R0, continuous_time_step, sim_time)

    # Interpolate I_diff to match the discrete time points
    I_diff_interp = np.interp(t_discrete, t_diff, I_diff)
    difference_percentage = (np.abs(I_discrete - I_diff_interp) / N) * 100

    # Plot results
    fig, axes = plt.subplots(2, 1, figsize=(8, 10))

    axes[0].plot(t_discrete, I_discrete, label='I(t) - Difference Equations', color='red', linestyle='dashed')
    axes[0].plot(t_diff, I_diff, label='I(t) - Differential Equations', color='blue')
    axes[0].set_title('Comparison of SEIRS Models with Seasonal Beta')
    axes[0].set_xlabel('Time (Days)')
    axes[0].set_ylabel('Infected Population')
    axes[0].legend()
    axes[0].grid()

    axes[1].plot(t_discrete, difference_percentage, label='Proportional Difference (%)', color='purple')
    axes[1].set_title('Relative Difference Between Models Over Time')
    axes[1].set_xlabel('Time (Days)')
    axes[1].set_ylabel('ΔI(t) / N (%)')
    axes[1].legend()
    axes[1].grid()

    plt.show()

ui = widgets.VBox([beta_base_slider, beta_variation_slider, sigma_slider, gamma_slider, omega_slider, N_slider, E0_slider, I0_slider, R0_slider, sim_time_slider, discrete_step_slider, continuous_step_slider])
out = widgets.interactive_output(plot_seirs, {
    'beta_base': beta_base_slider,
    'beta_variation': beta_variation_slider,
    'sigma': sigma_slider,
    'gamma': gamma_slider,
    'omega': omega_slider,
    'N': N_slider,
    'E0': E0_slider,
    'I0': I0_slider,
    'R0': R0_slider,
    'sim_time': sim_time_slider,
    'discrete_time_step': discrete_step_slider,
    'continuous_time_step': continuous_step_slider
})
display(ui, out)


Output()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.integrate import solve_ivp

# Seasonal beta function: higher in winter (southern hemisphere)
def beta_seasonal(t, beta_base, beta_variation):
    return beta_base * (1 + beta_variation * np.cos(2 * np.pi * (t / 365)))

# SEIRS Model with Frequency-Dependent Transmission (Equations of Differences)
def seirs_discrete(S0, E0, I0, R0, beta_base, beta_variation, sigma, gamma, omega, N, time_step, sim_time):
    steps = int(sim_time / time_step)
    S, E, I, R = [S0], [E0], [I0], [R0]
    time_points = np.linspace(0, sim_time, steps + 1)

    for i in range(steps):
        t = time_points[i]
        beta_t = beta_seasonal(t, beta_base, beta_variation)
        new_E = beta_t * (S[-1] * I[-1] / N) * time_step
        new_I = sigma * E[-1] * time_step
        new_R = gamma * I[-1] * time_step
        new_S = omega * R[-1] * time_step

        S.append(S[-1] - new_E + new_S)
        E.append(E[-1] + new_E - new_I)
        I.append(I[-1] + new_I - new_R)
        R.append(R[-1] + new_R - new_S)

    return np.array(I), time_points

# SEIRS Model with Frequency-Dependent Transmission (Equations of Differential)
def seirs_model(t, y, beta_base, beta_variation, sigma, gamma, omega, N):
    S, E, I, R = y
    beta_t = beta_seasonal(t, beta_base, beta_variation)
    dSdt = -beta_t * (S * I / N) + omega * R
    dEdt = beta_t * (S * I / N) - sigma * E
    dIdt = sigma * E - gamma * I
    dRdt = gamma * I - omega * R
    return [dSdt, dEdt, dIdt, dRdt]

def solve_seirs(beta_base, beta_variation, sigma, gamma, omega, N, S0, E0, I0, R0, time_step, sim_time):
    y0 = [S0, E0, I0, R0]
    t_span = [0, sim_time]
    t_eval = np.arange(0, sim_time + time_step, time_step)  # Avoid exceeding sim_time
    t_eval = t_eval[t_eval <= sim_time]  # Ensure t_eval is within bounds

    sol = solve_ivp(seirs_model, t_span, y0, args=(beta_base, beta_variation, sigma, gamma, omega, N), t_eval=t_eval)
    return sol.t, sol.y[2]

# Interactive Widgets
beta_base_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.3, description='Beta Base')
beta_variation_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.01, value=0.2, description='Beta Variation')
sigma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Sigma')
gamma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Gamma')
omega_slider = widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='Omega')
N_slider = widgets.IntSlider(min=100, max=10000, step=100, value=1000, description='Population')
E0_slider = widgets.IntSlider(min=1, max=500, step=1, value=10, description='Initial E')
I0_slider = widgets.IntSlider(min=0, max=500, step=1, value=10, description='Initial I')
R0_slider = widgets.IntSlider(min=0, max=500, step=1, value=0, description='Initial R')
sim_time_slider = widgets.IntSlider(min=50, max=3000, step=50, value=365, description='Simulation Time (days)')
discrete_step_slider = widgets.FloatSlider(min=0.05, max=1.0, step=0.05, value=0.5, description='Discrete Time Step')
continuous_step_slider = widgets.FloatSlider(min=0.02, max=2.0, step=0.02, value=0.1, description='Continuous Time Step')

def plot_seirs(beta_base, beta_variation, sigma, gamma, omega, N, E0, I0, R0, discrete_time_step, continuous_time_step, sim_time):
    S0 = N - E0 - I0 - R0

    # Solve the models
    I_discrete, t_discrete = seirs_discrete(S0, E0, I0, R0, beta_base, beta_variation, sigma, gamma, omega, N, discrete_time_step, sim_time)
    t_diff, I_diff = solve_seirs(beta_base, beta_variation, sigma, gamma, omega, N, S0, E0, I0, R0, continuous_time_step, sim_time)

    # Interpolate I_diff to match the discrete time points
    I_diff_interp = np.interp(t_discrete, t_diff, I_diff)
    difference_percentage = (np.abs(I_discrete - I_diff_interp) / N) * 100

    years = t_discrete / 365

    # Plot results
    fig, axes = plt.subplots(3, 1, figsize=(8, 15))

    axes[0].plot(t_discrete, I_discrete, label='I(t) - Difference Equations', color='red', linestyle='dashed')
    axes[0].plot(t_diff, I_diff, label='I(t) - Differential Equations', color='blue')
    axes[0].set_title('Comparison of SEIRS Models with Seasonal Beta')
    axes[0].set_xlabel('Time (Days)')
    axes[0].set_ylabel('Infected Population')
    axes[0].legend()
    axes[0].grid()

    axes[1].plot(t_discrete, difference_percentage, label='Proportional Difference (%)', color='purple')
    axes[1].set_title('Relative Difference Between Models Over Time')
    axes[1].set_xlabel('Time (Days)')
    axes[1].set_ylabel('ΔI(t) / N (%)')
    axes[1].legend()
    axes[1].grid()

    axes[2].plot(years, I_discrete, label='I(t) over Years', color='green')
    axes[2].set_title('Infected Population Over Years')
    axes[2].set_xlabel('Time (Years)')
    axes[2].set_ylabel('Infected Population')
    axes[2].legend()
    axes[2].grid()

    plt.show()

ui = widgets.VBox([beta_base_slider, beta_variation_slider, sigma_slider, gamma_slider, omega_slider, N_slider, E0_slider, I0_slider, R0_slider, sim_time_slider, discrete_step_slider, continuous_step_slider])
out = widgets.interactive_output(plot_seirs, {
    'beta_base': beta_base_slider,
    'beta_variation': beta_variation_slider,
    'sigma': sigma_slider,
    'gamma': gamma_slider,
    'omega': omega_slider,
    'N': N_slider,
    'E0': E0_slider,
    'I0': I0_slider,
    'R0': R0_slider,
    'sim_time': sim_time_slider,
    'discrete_time_step': discrete_step_slider,
    'continuous_time_step': continuous_step_slider
})
display(ui, out)


Output()